* Add small value to profit so we don't divide by 0


* GroupBy trade_id and sort ascending by bar_num for convenience
* Use recursive calculation of L_t = alpha * y_t + (1-alpha)*L_(t-1)
* This is very similar to what we spoke about with Nikos in recursive SQL queries
* Cache the df

# MSiA 431 - Big Data - Homework 3

## Kristiyan Dimitrov

In [27]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, countDistinct
from pyspark.sql import Window
from pyspark.sql.types import StructType, StructField, LongType, DoubleType
from pyspark.sql import functions as F
from pyspark.sql.functions import lit
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.sql.functions import col, weekofyear, year, month, window, count, lag, first, last, desc 
from operator import add
import pandas as pd
import numpy as np

In [2]:
spark = SparkSession.builder.appName('Problem 3').getOrCreate()

In [3]:
csv_path = 'data.csv'

In [4]:
df = spark.read.csv(csv_path, header = True , inferSchema = True, timestampFormat='YYYY-MM-DD HH:MM:SS a')

## DIVING IN

In [14]:
# Number of rows
df.count()

1082661

In [18]:
# This shows that there aren't any rows with missing values i.e. the data is clean!
df.na.drop().count()

1082661

In [21]:
# Number of unique values in each column
df.agg(*(countDistinct(col(c)).alias(c) for c in df.columns)).show()

+----------+-------+---------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|time_stamp|bar_num|direction|profit|var12|var13|var14|var15|var16|var17|var18|var23|var24|var25|var26|var27|var28|var34|var35|var36|var37|var38|var45|var46|var47|var48|var56|var57|var58|var67|var68|var78|trade_id|
+----------+-------+---------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|   1050263|    120|        2|  1957|   19|   18|   18|   18|   18|   18|   18|   18|   18|   18|   19|   18|   18|   18|   18|   18|   18|   18|   18|   18|   19|   18|   19|   18|   19|   18|   18|   19|   10652|
+----------+-------+---------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-

In [22]:
# we are told that the only relevant columns are time_stamp, bar_num, profit, trade_id and the vals columns. Therefore, the direction (probably selling short or long?) should be dropped
df = df.drop('direction')

In [35]:
print(df.dtypes)

[('time_stamp', 'timestamp'), ('bar_num', 'int'), ('profit', 'int'), ('var12', 'int'), ('var13', 'int'), ('var14', 'int'), ('var15', 'int'), ('var16', 'int'), ('var17', 'int'), ('var18', 'int'), ('var23', 'int'), ('var24', 'int'), ('var25', 'int'), ('var26', 'int'), ('var27', 'int'), ('var28', 'int'), ('var34', 'int'), ('var35', 'int'), ('var36', 'int'), ('var37', 'int'), ('var38', 'int'), ('var45', 'int'), ('var46', 'int'), ('var47', 'int'), ('var48', 'int'), ('var56', 'int'), ('var57', 'int'), ('var58', 'int'), ('var67', 'int'), ('var68', 'int'), ('var78', 'int'), ('trade_id', 'int')]


In [36]:
# Look at the data
df.show(150)

+-------------------+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|         time_stamp|bar_num|profit|var12|var13|var14|var15|var16|var17|var18|var23|var24|var25|var26|var27|var28|var34|var35|var36|var37|var38|var45|var46|var47|var48|var56|var57|var58|var67|var68|var78|trade_id|
+-------------------+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|2015-06-10 13:52:00|    120|  -142|   -8|   -8|   -5|    1|    1|    1|    1|   -9|    1|    1|    1|    1|    1|    1|    1|    1|    1|    1|    2|    3|    3|    3|    3|    3|    3|    8|    8|    9|     185|
|2015-06-10 13:51:00|    119|  -149|   -9|   -9|    1|    1|    1|    1|    1|   -5|    1|    1|    1|    1|    1|    1|    1|    1|    1|    1|

In [55]:
# TRIED TO SORT WITHIN GROUPBY BUT HAD ISSUES

# from pyspark.sql.functions import col, asc, sort_array, collect_list
# df.groupBy('trade_id').agg(sorted(col('bar_num'))).show()
# df.groupBy('trade_id').orderBy(asc('bar_num')).show()
# df.groupBy('trade_id').count().sort(col('bar_num')).show()

In [59]:
df.show()

+-------------------+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|         time_stamp|bar_num|profit|var12|var13|var14|var15|var16|var17|var18|var23|var24|var25|var26|var27|var28|var34|var35|var36|var37|var38|var45|var46|var47|var48|var56|var57|var58|var67|var68|var78|trade_id|
+-------------------+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|2015-06-10 13:52:00|    120|  -142|   -8|   -8|   -5|    1|    1|    1|    1|   -9|    1|    1|    1|    1|    1|    1|    1|    1|    1|    1|    2|    3|    3|    3|    3|    3|    3|    8|    8|    9|     185|
|2015-06-10 13:51:00|    119|  -149|   -9|   -9|    1|    1|    1|    1|    1|   -5|    1|    1|    1|    1|    1|    1|    1|    1|    1|    1|

In [60]:
df.agg({'time_stamp': 'min'}).show()

+-------------------+
|    min(time_stamp)|
+-------------------+
|2007-08-07 14:34:00|
+-------------------+



In [61]:
df.agg({'time_stamp': 'max'}).show()

+-------------------+
|    max(time_stamp)|
+-------------------+
|2015-08-03 11:15:00|
+-------------------+



### Removing all data in 2007

In [5]:
df_2008 = df.filter(df.time_stamp >= '2008-01-01 00:00:00')
df_2008.show()

+-------------------+-------+---------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|         time_stamp|bar_num|direction|profit|var12|var13|var14|var15|var16|var17|var18|var23|var24|var25|var26|var27|var28|var34|var35|var36|var37|var38|var45|var46|var47|var48|var56|var57|var58|var67|var68|var78|trade_id|
+-------------------+-------+---------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|2015-06-10 13:52:00|    120|       -1|  -142|   -8|   -8|   -5|    1|    1|    1|    1|   -9|    1|    1|    1|    1|    1|    1|    1|    1|    1|    1|    2|    3|    3|    3|    3|    3|    3|    8|    8|    9|     185|
|2015-06-10 13:51:00|    119|       -1|  -149|   -9|   -9|    1|    1|    1|    1|    1|   -5|    1|    

In [95]:
# Verifying that the last column is in 2008

from pyspark.sql import functions as F

expr = [F.last(col).alias(col) for col in df.columns]

df_2008.agg(*expr).show()

+-------------------+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|         time_stamp|bar_num|profit|var12|var13|var14|var15|var16|var17|var18|var23|var24|var25|var26|var27|var28|var34|var35|var36|var37|var38|var45|var46|var47|var48|var56|var57|var58|var67|var68|var78|trade_id|
+-------------------+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|2008-01-10 09:09:00|      1|   -20|    1|    2|    3|    3|    3|    3|    3|    3|    8|    8|    8|    8|    8|    8|    8|    8|    8|   -4|    9|    9|   -1|   -1|   -1|   -1|   -1|   -1|   -3|   -3|   10178|
+-------------------+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+

## Taking just the first 6 months of 2008

In [96]:
# Taking just the first 6 months of 2008
df_1 = df_2008.filter(df_2008.time_stamp <= '2008-06-01 00:00:00')
df_1.show()

+-------------------+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|         time_stamp|bar_num|profit|var12|var13|var14|var15|var16|var17|var18|var23|var24|var25|var26|var27|var28|var34|var35|var36|var37|var38|var45|var46|var47|var48|var56|var57|var58|var67|var68|var78|trade_id|
+-------------------+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|2008-04-10 04:19:00|    120|  -131|    9|    7|    7|    7|    7|    9|    9|    6|    6|    6|    7|    7|    7|    6|    6|    7|    9|    9|    9|    9|    9|    9|    9|    9|    9|   -1|    6|    6|    9853|
|2008-04-10 04:18:00|    119|   -97|    3|    2|    2|    2|    2|    3|    3|    1|    1|    1|    1|    2|    2|    1|    1|    2|    3|    3|

In [97]:
# Confirming the earliest row
df_1.agg(*expr).show()

+-------------------+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|         time_stamp|bar_num|profit|var12|var13|var14|var15|var16|var17|var18|var23|var24|var25|var26|var27|var28|var34|var35|var36|var37|var38|var45|var46|var47|var48|var56|var57|var58|var67|var68|var78|trade_id|
+-------------------+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|2008-01-10 09:09:00|      1|   -20|    1|    2|    3|    3|    3|    3|    3|    3|    8|    8|    8|    8|    8|    8|    8|    8|    8|   -4|    9|    9|   -1|   -1|   -1|   -1|   -1|   -1|   -3|   -3|   10178|
+-------------------+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+

In [98]:
# Looking at the latest row - looks plausible, the trades in the data are kind of sparse
expr = [F.first(col).alias(col) for col in df.columns]
df_1.agg(*expr).show()

+-------------------+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|         time_stamp|bar_num|profit|var12|var13|var14|var15|var16|var17|var18|var23|var24|var25|var26|var27|var28|var34|var35|var36|var37|var38|var45|var46|var47|var48|var56|var57|var58|var67|var68|var78|trade_id|
+-------------------+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|2008-04-10 04:19:00|    120|  -131|    9|    7|    7|    7|    7|    9|    9|    6|    6|    6|    7|    7|    7|    6|    6|    7|    9|    9|    9|    9|    9|    9|    9|    9|    9|   -1|    6|    6|    9853|
+-------------------+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+

In [99]:
# Apperently not all trades have all 120 bars
max_bar_per_trade = df_1.groupBy(col("trade_id")).agg({"bar_num": "max"}).alias('max_bar_num')
max_bar_per_trade.show()
print(max_bar_per_trade.count())

+--------+------------+
|trade_id|max(bar_num)|
+--------+------------+
|    9900|         120|
|    9852|         120|
|   10081|         120|
|    9879|          66|
|   10121|         120|
|    9946|         120|
|   10032|         120|
|    9775|         120|
|    9914|         120|
|   10090|         120|
|   10128|         120|
|    9721|         120|
|   10013|         120|
|    9925|         120|
|    9731|          63|
|   10143|         120|
|   10183|         120|
|    9867|         120|
|   10195|         120|
|    9898|         120|
+--------+------------+
only showing top 20 rows

529


In [100]:
max_bar_per_trade.filter(max_bar_per_trade['max(bar_num)'] == 120).show()
print(max_bar_per_trade.filter(max_bar_per_trade['max(bar_num)'] == 120).count())

+--------+------------+
|trade_id|max(bar_num)|
+--------+------------+
|    9900|         120|
|    9852|         120|
|   10081|         120|
|   10121|         120|
|    9946|         120|
|   10032|         120|
|    9775|         120|
|    9914|         120|
|   10090|         120|
|   10128|         120|
|    9721|         120|
|   10013|         120|
|    9925|         120|
|   10143|         120|
|   10183|         120|
|    9867|         120|
|   10195|         120|
|    9898|         120|
|   10022|         120|
|    9968|         120|
+--------+------------+
only showing top 20 rows

381


### It seems there are 381 trades with all 120 bars out of 529 trades in the first 6 months of data

In [107]:
max_bar = max_bar_per_trade.agg({"max(bar_num)": "max"}).collect()[0][0]
max_bar

120

### Ok, so let's try this - for each group, I want to calculate the profit EWMA up until now, but excluding the last (columns_up_until_now % 10) rows

In [108]:
df_1.show(3)

+-------------------+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|         time_stamp|bar_num|profit|var12|var13|var14|var15|var16|var17|var18|var23|var24|var25|var26|var27|var28|var34|var35|var36|var37|var38|var45|var46|var47|var48|var56|var57|var58|var67|var68|var78|trade_id|
+-------------------+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|2008-04-10 04:19:00|    120|  -131|    9|    7|    7|    7|    7|    9|    9|    6|    6|    6|    7|    7|    7|    6|    6|    7|    9|    9|    9|    9|    9|    9|    9|    9|    9|   -1|    6|    6|    9853|
|2008-04-10 04:18:00|    119|   -97|    3|    2|    2|    2|    2|    3|    3|    1|    1|    1|    1|    2|    2|    1|    1|    2|    3|    3|

In [125]:
# Let's take a subset of columns for ease
df_subset = df_1.select(['bar_num', 'profit', 'trade_id'])
df_subset.show(3)

+-------+------+--------+
|bar_num|profit|trade_id|
+-------+------+--------+
|    120|  -131|    9853|
|    119|   -97|    9853|
|    118|  -124|    9853|
+-------+------+--------+
only showing top 3 rows



In [126]:
# DON'T FORGET TO CACHE THE DATAFRAME
df_subset.cache()

DataFrame[bar_num: int, profit: int, trade_id: int]

In [129]:
df_subset.show()

+-------+------+--------+
|bar_num|profit|trade_id|
+-------+------+--------+
|    120|  -131|    9853|
|    119|   -97|    9853|
|    118|  -124|    9853|
|    117|  -102|    9853|
|    116|  -139|    9853|
|    115|   -96|    9853|
|    114|   -86|    9853|
|    113|   -92|    9853|
|    112|   -82|    9853|
|    111|  -141|    9853|
|    110|   -78|    9853|
|    109|   -67|    9853|
|    108|   -68|    9853|
|    107|  -126|    9853|
|    106|   -96|    9853|
|    105|  -116|    9853|
|    104|  -122|    9853|
|    103|  -183|    9853|
|    102|  -162|    9853|
|    101|  -164|    9853|
+-------+------+--------+
only showing top 20 rows



## Trying simpler method with changing window?

In [ ]:
w = Window.partitionBy("trade_id").orderBy("bar_num")

## Tucker's method

In [137]:
# Let's try and do Tucker's approach !!!; REMEMBER: Use recursive calculation of L_t = alpha * y_t + (1-alpha)*L_(t-1)

# First define a schema for intermediate results
schema = StructType([StructField('bar_num', LongType(), False), #placeholder for transaction id
                     StructField('trade_id', LongType(), False), #placeholder for coupon id
                     StructField('profit_ewma', DoubleType(), True)]) #where we will hold the sum of rpevious deductions
                     
#                      StructField('L_total', DoubleType(), True)]) #the actual discount for thsi program at this id

results = spark.createDataFrame([], schema)

In [138]:
schema.fieldNames()

['bar_num', 'trade_id', 'profit_ewma']

In [139]:
results.show()

+-------+--------+-----------+
|bar_num|trade_id|profit_ewma|
+-------+--------+-----------+
+-------+--------+-----------+



In [140]:
df_subset.filter(f'bar_num ==1').count()

529

In [134]:
df_subset.filter(f'bar_num ==1').show()

+-------+------+--------+
|bar_num|profit|trade_id|
+-------+------+--------+
|      1|   -34|    9853|
|      1|   -29|    9913|
|      1|    -3|   10096|
|      1|   -60|   10145|
|      1|     5|    9679|
|      1|   -44|    9957|
|      1|   -20|   10018|
|      1|   -26|    9707|
|      1|    -4|    9816|
|      1|   -21|    9738|
|      1|   -60|    9902|
|      1|   -26|   10003|
|      1|   -17|    9793|
|      1|   -35|    9894|
|      1|   -14|    9920|
|      1|    -8|   10128|
|      1|   -60|    9727|
|      1|     1|   10045|
|      1|   -20|   10124|
|      1|   -10|   10158|
+-------+------+--------+
only showing top 20 rows



In [151]:
# CONFIGURATION
alpha = 0.2

In [149]:
# Test 1 with rdd
df_test = df_subset.filter(f'bar_num ==1')
lst_test = df_test.rdd.map(lambda x: x.profit).collect()
lst_test[:10]

[-34, -29, -3, -60, 5, -44, -20, -26, -4, -21]

In [156]:
ewma_lst(alpha, lst_test)

-26.47032272251052

## Test 2 with collect_list - Trying to take a subset of the cached dataframe and manipulate it in such a way as to get the ewma for each trade_id

In [159]:
# First I filter to the wanted subset
df_test_2 = df_subset.filter(f'bar_num <= 23')
df_test_2.show()

+-------+------+--------+
|bar_num|profit|trade_id|
+-------+------+--------+
|     23|    85|    9853|
|     22|    45|    9853|
|     21|    29|    9853|
|     20|    65|    9853|
|     19|    40|    9853|
|     18|    48|    9853|
|     17|    55|    9853|
|     16|    32|    9853|
|     15|    21|    9853|
|     14|    32|    9853|
|     13|     5|    9853|
|     12|    32|    9853|
|     11|   -26|    9853|
|     10|   -16|    9853|
|      9|    16|    9853|
|      8|    -3|    9853|
|      7|   -85|    9853|
|      6|   -67|    9853|
|      5|   -67|    9853|
|      4|   -55|    9853|
+-------+------+--------+
only showing top 20 rows



In [170]:
## This is the EWMA function, which takes a list of values and an alpha parameters and calculates the exponentially weighted average
## Testing EWMA with a range of values
def ewma_lst(alpha, lst):
    
    res = 0
    
    for ii in range(len(lst)):
        res += alpha * ( (1-alpha)**ii ) * lst[ii]
    
    return res

In [181]:
# Collecting all the profits for a given trade_id in one place
df_intermediate_1 = df_test_2.groupby('trade_id').agg(F.collect_list('profit'))
df_intermediate_1.show(4, False)

+--------+--------------------------------------------------------------------------------------------------------------------------+
|trade_id|collect_list(profit)                                                                                                      |
+--------+--------------------------------------------------------------------------------------------------------------------------+
|9900    |[-24, -18, -18, -8, -3, 6, 7, 17, 17, 19, 28, 28, 17, 2, 8, 9, 30, 40, 47, 47, 5, -5, -20]                                |
|9852    |[-158, -135, -131, -100, -101, -73, -92, -50, -70, -80, -65, -69, -81, -51, -40, -41, -72, -94, -62, -32, -34, -9, -26]   |
|10081   |[60, 64, 60, 43, 21, 47, 32, 32, 74, 84, 82, 79, 79, 90, 50, 94, 40, 24, -6, 2, -1, -5, -25]                              |
|9879    |[-94, -99, -119, -124, -144, -149, -159, -93, -102, -104, -97, -73, -74, -84, -89, -74, -82, -55, -53, -51, -61, -33, -35]|
+--------+----------------------------------------------------

In [180]:
# Verifying what value I should expect to get for trade_id 9900
test_lst = [-24, -18, -18, -8, -3, 6, 7, 17, 17, 19, 28, 28, 17, 2, 8, 9, 30, 40, 47, 47, 5, -5, -20]
ewma_lst(alpha, test_lst)

-6.39696553721959

In [182]:
# Verifying what value I should expect to get for trade_id 9852
test_lst_2 = [-158, -135, -131, -100, -101, -73, -92, -50, -70, -80, -65, -69, -81, -51, -40, -41, -72, -94, -62, -32, -34, -9, -26] 
ewma_lst(alpha, test_lst_2)

-110.85622541117694

In [172]:
# Register UDF
ewma = spark.udf.register("ewma_lst", ewma_lst)

In [184]:
# Apply that UDF to the collected profits
new_df = df_intermediate_1.select('trade_id', ewma(F.lit(alpha), col("collect_list(profit)")))
new_df.show()

+--------+-----------------------------------+
|trade_id|ewma_lst(0.2, collect_list(profit))|
+--------+-----------------------------------+
|    9900|                  -6.39696553721959|
|    9852|                -110.85622541117694|
|   10081|                  53.44523297764724|
|    9879|                -110.46396394406825|
|   10121|                -35.662528520782246|
|    9946|                 18.690290738956254|
|   10032|                 41.875904055260776|
|    9775|                 123.86575885095068|
|    9914|                 160.42033722775304|
|   10090|                 -79.14705391489693|
|   10128|                 17.392192870696775|
|    9721|                  44.49860409721482|
|   10013|                  32.80738393926695|
|    9925|                  289.8594115150316|
|    9731|                  67.04531368634817|
|   10143|                -121.21383715447581|
|   10183|                 -69.79943489368077|
|    9867|                 -70.35492962433317|
|   10195|   

### GREAT! I get the ewma for the specific subset!

Now, say ii = 33 i.e. we are looking at bar 33 and below; I want to just add bar 33 as an additional column

In [186]:
iii = 33

In [187]:
final_df = new_df.withColumn('bar_num', lit(iii))
final_df.show()

+--------+-----------------------------------+-------+
|trade_id|ewma_lst(0.2, collect_list(profit))|bar_num|
+--------+-----------------------------------+-------+
|    9900|                  -6.39696553721959|     33|
|    9852|                -110.85622541117694|     33|
|   10081|                  53.44523297764724|     33|
|    9879|                -110.46396394406825|     33|
|   10121|                -35.662528520782246|     33|
|    9946|                 18.690290738956254|     33|
|   10032|                 41.875904055260776|     33|
|    9775|                 123.86575885095068|     33|
|    9914|                 160.42033722775304|     33|
|   10090|                 -79.14705391489693|     33|
|   10128|                 17.392192870696775|     33|
|    9721|                  44.49860409721482|     33|
|   10013|                  32.80738393926695|     33|
|    9925|                  289.8594115150316|     33|
|    9731|                  67.04531368634817|     33|
|   10143|

In [199]:
final_df.dtypes

[('trade_id', 'int'),
 ('ewma_lst(0.2, collect_list(profit))', 'string'),
 ('bar_num', 'int')]

In [198]:
results.union(final_df).show()

+--------+-------------------+-------+
|trade_id|        profit_ewma|bar_num|
+--------+-------------------+-------+
|    9900|  -6.39696553721959|     33|
|    9852|-110.85622541117694|     33|
|   10081|  53.44523297764724|     33|
|    9879|-110.46396394406825|     33|
|   10121|-35.662528520782246|     33|
|    9946| 18.690290738956254|     33|
|   10032| 41.875904055260776|     33|
|    9775| 123.86575885095068|     33|
|    9914| 160.42033722775304|     33|
|   10090| -79.14705391489693|     33|
|   10128| 17.392192870696775|     33|
|    9721|  44.49860409721482|     33|
|   10013|  32.80738393926695|     33|
|    9925|  289.8594115150316|     33|
|    9731|  67.04531368634817|     33|
|   10143|-121.21383715447581|     33|
|   10183| -69.79943489368077|     33|
|    9867| -70.35492962433317|     33|
|   10195|   9.51560103151829|     33|
|    9898| 15.823557009260702|     33|
+--------+-------------------+-------+
only showing top 20 rows



## Putting it all together in a for loop

In [6]:
# CONFIGURATION
alpha = .2

In [17]:
## This is the EWMA function, which takes a list of values and an alpha parameters and calculates the exponentially weighted average
## Testing EWMA with a range of values
def ewma_lst(alpha, lst):
    
    res = 0
    
    for ii in range(len(lst)):
        res += alpha * ( (1-alpha)**ii ) * lst[ii]
    
    return res

In [18]:
# Register UDF
ewma = spark.udf.register("ewma_lst", ewma_lst)

In [19]:
# Taking only data after 2008, before that is not relevant
df_2008 = df.filter(df.time_stamp >= '2008-01-01 00:00:00')

# Dropping 'direction' column
df_2008 = df_2008.drop('direction')

# Taking first chunk of data; NEED TO PARAMETRIZE WITH YEAR & MONTH LATER <------------------ <------------------ <------------------ <------------------ <------------------
df_1 = df_2008.filter(df_2008.time_stamp <= '2008-06-01 00:00:00')

# Let's take a subset of columns for ease
df_subset = df_1.select(['bar_num', 'profit', 'trade_id'])

In [20]:
# Verifying the max bar value across all trades
max_bar_per_trade = df_1.groupBy(col("trade_id")).agg({"bar_num": "max"}).alias('max_bar_num')
max_bar = max_bar_per_trade.agg({"max(bar_num)": "max"}).collect()[0][0]
max_bar

120

In [21]:
# Create schema for empty dataframe, which will hold all the calculated ewma profits
schema = StructType([StructField('trade_id', LongType(), False),
                     StructField('profit_ewma', DoubleType(), False), 
                     StructField('bar_num', LongType(), True)])

results = spark.createDataFrame([], schema)

In [22]:
df_subset.cache()

DataFrame[bar_num: int, profit: int, trade_id: int]

In [23]:
# The goal of this for loop is to calculate a feature based on profit

for ii in range(11, max_bar): # For bars 1, 2, 3, ... 10, we don't need to do anything; So, when we do left join, those feature values for bars 1-10 should be null
    
#     print(f'Start {ii}')
    
    if ii % 10 == 0: # This means we are in the situation of taking bar 20, 30, 40, etc.
        bars_to_take = ii - 10 # For bar 20, we want bars 10 and below; for bar 30, we want bars 20 and below...
    else:
        bars_to_take = ii - ii%10 # E.g. if we are at bar 33, we want bars 33 - 3 = 30 and below
    
    # Taking only the part of the dataset, which contains the subset of bars we are interested in
    df_filtered = df_subset.filter(f'bar_num <= {bars_to_take}')
        
    # Collecting all the profits for a given trade_id in one place (list with its corresponding trade_id)
    df_intermediate = df_filtered.groupby('trade_id').agg(F.collect_list('profit'))
        
    # Apply the UDF EWMA function to the collected list of profits
    df_ewma = df_intermediate.select('trade_id', ewma(F.lit(alpha), col("collect_list(profit)")))
        
    # Adding the bar_num we are currently at, so we can properly join later
    final_df = df_ewma.withColumn('bar_num', lit(ii))
        
    results = results.union(final_df)   
    
#     print(f'End {ii}')
    
    

In [14]:
results.show(10)

+--------+-------------------+-------+
|trade_id|        profit_ewma|bar_num|
+--------+-------------------+-------+
|    9900| 14.230217728000003|     11|
|    9852|-45.088219033600005|     11|
|   10081|  42.92491776000001|     11|
|    9879| -64.16702259200002|     11|
|   10121|-16.785674240000006|     11|
|    9946|  9.038351359999998|     11|
|   10032| 15.545000038400003|     11|
|    9775| 28.437628416000003|     11|
|    9914|  93.61254041600002|     11|
|   10090|      -37.236989952|     11|
+--------+-------------------+-------+
only showing top 10 rows



In [21]:
# results.filter(results.trade_id == 9900).show() # Tried to let this run for ~ 10 minutes, but it didn't finish; aborting and hoping this works on distributed

In [ ]:
# Now I need to join the original dataframe with the results i.e. to add the profit_ewma feature 

In [24]:
df_1.show()

+-------------------+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|         time_stamp|bar_num|profit|var12|var13|var14|var15|var16|var17|var18|var23|var24|var25|var26|var27|var28|var34|var35|var36|var37|var38|var45|var46|var47|var48|var56|var57|var58|var67|var68|var78|trade_id|
+-------------------+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|2008-04-10 04:19:00|    120|  -131|    9|    7|    7|    7|    7|    9|    9|    6|    6|    6|    7|    7|    7|    6|    6|    7|    9|    9|    9|    9|    9|    9|    9|    9|    9|   -1|    6|    6|    9853|
|2008-04-10 04:18:00|    119|   -97|    3|    2|    2|    2|    2|    3|    3|    1|    1|    1|    1|    2|    2|    1|    1|    2|    3|    3|

In [25]:
df_with_new_column = df_1.join(results, on = ['trade_id', 'bar_num'])

In [26]:
df_with_new_column.show()

Py4JJavaError: An error occurred while calling o4731.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Total size of serialized results of 20437 tasks (1024.0 MB) is bigger than spark.driver.maxResultSize (1024.0 MB)
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1891)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1879)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1878)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1878)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:927)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(

## Step-By-Step of above for-loop 

In [225]:
bars_to_take = 10

In [226]:
df_subset.show(3)

+-------+------+--------+
|bar_num|profit|trade_id|
+-------+------+--------+
|    120|  -131|    9853|
|    119|   -97|    9853|
|    118|  -124|    9853|
+-------+------+--------+
only showing top 3 rows



In [227]:
df_filtered = df_subset.filter(f'bar_num <= {bars_to_take}')
df_filtered.show(3)

+-------+------+--------+
|bar_num|profit|trade_id|
+-------+------+--------+
|     10|   -16|    9853|
|      9|    16|    9853|
|      8|    -3|    9853|
+-------+------+--------+
only showing top 3 rows



In [228]:
df_intermediate = df_filtered.groupby('trade_id').agg(F.collect_list('profit'))
df_intermediate.show(3)

+--------+--------------------+
|trade_id|collect_list(profit)|
+--------+--------------------+
|    9900|[2, 8, 9, 30, 40,...|
|    9852|[-51, -40, -41, -...|
|   10081|[90, 50, 94, 40, ...|
+--------+--------------------+
only showing top 3 rows



In [229]:
df_ewma = df_intermediate.select('trade_id', ewma(F.lit(alpha), col("collect_list(profit)")))
df_ewma.show(3)

+--------+-----------------------------------+
|trade_id|ewma_lst(0.2, collect_list(profit))|
+--------+-----------------------------------+
|    9900|                 14.230217728000003|
|    9852|                -45.088219033600005|
|   10081|                  42.92491776000001|
+--------+-----------------------------------+
only showing top 3 rows



In [230]:
final_df = df_ewma.withColumn('bar_num', lit(ii))
final_df.show(3)

+--------+-----------------------------------+-------+
|trade_id|ewma_lst(0.2, collect_list(profit))|bar_num|
+--------+-----------------------------------+-------+
|    9900|                 14.230217728000003|    119|
|    9852|                -45.088219033600005|    119|
|   10081|                  42.92491776000001|    119|
+--------+-----------------------------------+-------+
only showing top 3 rows



In [231]:
# Create schema for empty dataframe, which will hold all the calculated ewma profits
schema = StructType([StructField('trade_id', LongType(), False),
                     StructField('profit_ewma', DoubleType(), False), 
                     StructField('bar_num', LongType(), True)])

results = spark.createDataFrame([], schema)
results.show(3)

+--------+-----------+-------+
|trade_id|profit_ewma|bar_num|
+--------+-----------+-------+
+--------+-----------+-------+



In [232]:
results = results.union(final_df)
results.show()

+--------+-------------------+-------+
|trade_id|        profit_ewma|bar_num|
+--------+-------------------+-------+
|    9900| 14.230217728000003|    119|
|    9852|-45.088219033600005|    119|
|   10081|  42.92491776000001|    119|
|    9879| -64.16702259200002|    119|
|   10121|-16.785674240000006|    119|
|    9946|  9.038351359999998|    119|
|   10032| 15.545000038400003|    119|
|    9775| 28.437628416000003|    119|
|    9914|  93.61254041600002|    119|
|   10090|      -37.236989952|    119|
|   10128| 22.083040563200004|    119|
|    9721|      -18.770504704|    119|
|   10013| 27.960904192000005|    119|
|    9925| 220.04457932800008|    119|
|    9731|-10.384396083199999|    119|
|   10143| -96.87878338560002|    119|
|   10183| -37.69082163200001|    119|
|    9867|-35.009529241600006|    119|
|   10195|-10.397627392000006|    119|
|    9898|  94.27531079680001|    119|
+--------+-------------------+-------+
only showing top 20 rows



In [233]:
results.count()

529

In [ ]:
for i in range(1,max_value):
    level = df.filter(f"value = {i}") #one filter on the cached dataframe

    if i>1:
        level = level.drop('previous_deductions')
        level = level.join(next_level, on = ['idx', 'level']) #you will see this later, it has values for previous_deductions
    
    apply_discounts = level.withColumn('coupon_discount', expr("discount_percent * (Cost - previous_deductions)"))

    #The next two pieces are only applicable to this exact example, but explain how to pass information between levels efficiently
    this_level = (apply_discounts.select('idx', 'coupon_discount', 'previous_deductions')
                                 .groupBy('idx', 'previous_deductions').agg(sum('coupon_discount').alias('coupon_discount')))
                                 #This might nto be applicable to you, but the idea here is that multiple coupons can apply at a given level
    
    next_level = (this_level.withColumn('sum_deductions', expr("previous_deductions + coupon_discount"))
                            .withColumn('level', lit(i+1)))
                            # This is a trick to pass values to the next level
                            # If there is no next level, this is still a valid method, because next_level will not join on null values

    #finally we select the desired values and union the schema
    explicit discounts = apply_discounts.select('idx', 'coupon_id', 'previous_deductions', 'coupon_discount')
    #be careful with plain union. It is more efficient than other union methods, but it is schema agnostic
    results = results.union(explicit_discounts)


df_with_new_column = df.join(results, on = ['idx', 'coupon_id'])
#tada!
df_woth_new_column.write(...) #actually executes

In [ ]:
# APPENDIX

apply_ewma_step = level.withColumn('L_total', expr(' {alpha}*profit + (1-{alpha})*L_previous'))

### VErifying that there are values with profit = 0

In [26]:
df.filter(df.profit == 0).show()

+-------------------+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|         time_stamp|bar_num|profit|var12|var13|var14|var15|var16|var17|var18|var23|var24|var25|var26|var27|var28|var34|var35|var36|var37|var38|var45|var46|var47|var48|var56|var57|var58|var67|var68|var78|trade_id|
+-------------------+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+
|2015-05-20 09:36:00|      2|     0|   -6|   -6|   -7|   -9|   -9|   -9|   -9|   -9|   -8|   -8|   -9|   -9|   -9|   -8|   -8|   -8|   -9|   -9|   -9|   -9|    1|   -9|    1|    1|    1|    3|   -6|   -6|     263|
|2015-04-01 06:34:00|     12|     0|    4|    4|    4|    4|    4|    4|   -8|    1|    1|    1|    3|    3|   -6|    1|    3|    8|   -4|   -4|

In [27]:
# Testing EWMA
def ewma(alpha, n):
    
    res = 0
    
    for ii in range(n):
        res += alpha * ( (1-alpha)**ii )
    
    return res
    

In [30]:
# Correct!
ewma(.2, 3)

0.4880000000000001

In [169]:
## Testing EWMA with a range of values
def ewma_lst(alpha, lst):
    
    res = 0
    
    for ii in range(len(lst)):
        res += alpha * ( (1-alpha)**ii ) * lst[ii]
    
    return res

In [33]:
# Correct!
ewma_2(.2, [1,1,1])

0.4880000000000001